# Алгоритм Томпсона на батчах (без контекста)

1. На первом батче распределяем юзеров 50% на 50%.
2. Вероятность конверсии каждого варианта распределена по Beta распределению с $\alpha=1$,
$\beta=1$.
3. В конце каждого батча пересчитываем вероятности превосходства по точной формуле,
взятой отсюда https://www.johndcook.com//UTMDABTR-005-05.pdf
4. Распределяем трафик в пропорции вероятностей превосходства для каждого варианта
5. Останавливаем эксперимент при достижении определенной вероятности превосходства,
но не раньше определенного дня, чтобы учесть календарные факторы

Потенциальные проблемы:
- слишком рано отдаем трафик победителю
- из-за дисбаланса распределения трафика может быть больше успешных конверсий в этом варианте
(можно попробовать применить нормализацию)


***Реализуем алгоритм***

0. **Инициализация** - *BatchThompson(n_arms)*. Аргумент на вход: число вариантов сплита.
Здесь также инициализируются массивы для параметров Бета-распределений и вероятность превосходства = 0.5.

1. **Метод сплита** - *split_data()*. Исходя из вероятности превосходства вычисляем сплит по вариантам.
Возвращаем данные по конверсии на текущем батче для пересчета Бета-распределений.

2. **Метод изменения параметров распределения** - *.update_beta_params(data)*. Аргументы на вход: numpy массив со значениями конверсии по
каждому варианту. В случае неравномерного распределения по вариантам ставятся пропуски.
 - Проверяем, чтобы число столбцов совпадало с числом вариантов из инициализации.
 - Суммируем нули и единицы и обновляем параметры

3. **Метод пересчета** - *update_prob_super()*. Аргументов нет, так как учитывает измененные параметры
$\alpha$ (накопленное число успешных конверсий) и
$\beta$ (накопленное число неудачных конверсий) для всех вариантов.
 - Считаем по точной формуле
 - Выдаем массив из вероятностей превосходства

4. **Вероятность превосходства** - *prob_super_tuple()*. Аргументов нет, так как берем пересчитанные параметры.
5. **Критерий остановки** (*stopping_criterion*) - условия цикла while. Либо вероятность превосходства выше заданной
величины, либо закончились наблюдения.

In [32]:
import os
import numpy as np
from typing import List, Tuple
from scipy.stats import beta

# Функции для вычисления вероятности превосходства по точной формуле
from math import lgamma
from numba import jit

# Графики
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


def create_directory_plot(folder, file_name):
    directory_plots = 'Plot/Thompson/' + folder + "/"
    try:
        os.mkdir(directory_plots)
    except:
        pass
    beta_distr_plot = PdfPages(directory_plots + file_name + ".pdf")
    return beta_distr_plot


@jit
def h(a, b, c, d):
    num = lgamma(a + c) + lgamma(b + d) + lgamma(a + b) + lgamma(c + d)
    den = lgamma(a) + lgamma(b) + lgamma(c) + lgamma(d) + lgamma(a + b + c + d)
    return np.exp(num - den)

@jit
def g0(a, b, c):
    return np.exp(lgamma(a + b) + lgamma(a + c) - (lgamma(a + b + c) + lgamma(a)))

@jit
def hiter(a, b, c, d):
    while d > 1:
        d -= 1
        yield h(a, b, c, d) / d

def g(a, b, c, d):
    return g0(a, b, c) + sum(hiter(a, b, c, d))

def calc_prob_between(alphas, bethas):
    return g(alphas[0], bethas[0], alphas[1], bethas[1])


class BatchThompson:
    def __init__(self, n_arms: np.uint8):
        self._n_arms = n_arms
        self._alphas = np.repeat(1.0, n_arms)
        self._bethas = np.repeat(1.0, n_arms)
        self._prob_superiority_tuple = (0.5, 0.5)

    def split_data_historic(self, data_experiment: np.array, cumulative_observs: List, batch_split_obs: List):
        """
        Split data in every batch iteration
        :param data_experiment: historic data
        :param cumulative_observs: list with cumulative observations for every arm
        :param batch_split_obs: how many observation we must extract this iter
        :return:
        """
        n_rows, n_cols = np.max(batch_split_obs), self._n_arms
        data_split = np.empty((n_rows, n_cols))
        data_split[:] = np.nan
        for i in range(data_experiment.shape[1]):
            data_split[:batch_split_obs[i], i] = \
                data_experiment[cumulative_observs[i] : cumulative_observs[i] + batch_split_obs[i], i]
        return data_split



    def split_data_random(self, **kwargs):
        """

        :param kwargs: changeable params for generation samples
        :return:
        """
        ...


    def update_beta_params(self, batch_data: np.array):
        assert data.shape[1] == self._n_arms

        self._alphas += np.nansum(batch_data, axis=0)
        self._bethas += np.sum(batch_data == 0, axis=0)
        return self._alphas, self._bethas

    def update_prob_super(self, method_calc) -> Tuple:
        if method_calc == 'integrating':
            prob_superiority =  calc_prob_between(self._alphas, self._bethas)
            self._prob_superiority_tuple = (prob_superiority, 1 - prob_superiority)
            return self._prob_superiority_tuple
    def create_plots(self, beta_distr_plot):
        x = np.linspace(0, 1, 100)
        rv1 = beta(self._alphas[0], self._bethas[0])
        rv2 = beta(self._alphas[1], self._bethas[1])
        fix, ax = plt.subplots()
        ax.plot(x, rv1.pdf(x), label='control')
        ax.plot(x, rv2.pdf(x), label='testing')
        leg = ax.legend();
        plt.title(f"Вероятность превосходства в %: "
                  f"{np.round(tuple(map(lambda x: x * 100, self._prob_superiority_tuple)), 1)}")
        beta_distr_plot.savefig()
        plt.close()
# Experiment params
n_obs, n_arms = 10000, 2
batch_size = 100
probability_superiority =  0.5



# Generating data
data = np.empty(shape=(n_obs, n_arms))
np.random.seed(1)
data[:, 0] = np.random.binomial(n=1, p=0.4, size=n_obs)
data[:, 1] = np.random.binomial(n=1, p=0.45, size=n_obs)

batchT = BatchThompson(n_arms=2)
probability_superiority = batchT.update_prob_super(method_calc="integrating") # recalculate shares
cumulative_observs = np.repeat(0, n_arms)  # how many observations we extract every iter for every arm
stopping_criterion = (np.max(probability_superiority) <= 0.999) & (np.max(cumulative_observs) <  n_obs - 2 * batch_size)

# Plots
folder, file_name = "Experiment2", "medium_difference_conv"
beta_distr_plot = create_directory_plot(folder, file_name)
while stopping_criterion:
    batch_split_obs = np.round(np.array(batch_size) * probability_superiority).astype(np.uint16)  # get number of observations every arm
    batch_data = batchT.split_data_historic(data_experiment=data, cumulative_observs=cumulative_observs,
                                            batch_split_obs=batch_split_obs)
    alphas_batch, bethas_batch = batchT.update_beta_params(batch_data)  # update beta distributions
    cumulative_observs += batch_split_obs  # cumulative sum of observations for every arm
    probability_superiority = batchT.update_prob_super(method_calc="integrating") # recalculate shares
    print(f"Вероятность превосходства на каждом шаге в %: "
          f"{np.round([i * 100 for i in probability_superiority], 1)}")
    batchT.create_plots(beta_distr_plot)
beta_distr_plot.close()

Вероятность превосходства на каждом шаге в %: [15.9 84.1]
Вероятность превосходства на каждом шаге в %: [ 0.4 99.6]
Вероятность превосходства на каждом шаге в %: [ 1.2 98.8]
Вероятность превосходства на каждом шаге в %: [ 0.9 99.1]
Вероятность превосходства на каждом шаге в %: [ 1.5 98.5]
Вероятность превосходства на каждом шаге в %: [ 2.1 97.9]
Вероятность превосходства на каждом шаге в %: [ 2.6 97.4]
Вероятность превосходства на каждом шаге в %: [ 2.1 97.9]
Вероятность превосходства на каждом шаге в %: [ 3. 97.]
Вероятность превосходства на каждом шаге в %: [ 3.8 96.2]
Вероятность превосходства на каждом шаге в %: [ 7.1 92.9]
Вероятность превосходства на каждом шаге в %: [11.1 88.9]
Вероятность превосходства на каждом шаге в %: [ 8.3 91.7]
Вероятность превосходства на каждом шаге в %: [19.7 80.3]
Вероятность превосходства на каждом шаге в %: [17. 83.]
Вероятность превосходства на каждом шаге в %: [ 7.7 92.3]
Вероятность превосходства на каждом шаге в %: [ 4.7 95.3]
Вероятность превос

ValueError: could not broadcast input array from shape (19,) into shape (99,)

In [30]:
from scipy import stats
from AB_classic import get_size_zratio
get_size_zratio(0.4, 0.45, 0.05, 0.2)

1530

In [33]:
beta_distr_plot.close()

In [113]:
(0.35 - 0.3) / 0.3


0.16666666666666663